In [2]:
import re
import pandas as pd
import numpy as np
from nltk.tokenize import WordPunctTokenizer
from gensim.models import Word2Vec
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.ensemble import VotingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.feature_extraction.text import CountVectorizer

In [3]:
data = pd.read_csv("data/labeled.csv")

data.head()

,comment,toxic
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0
2,Собаке - собачья смерть\n,1.0
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0


In [4]:
stop_words = set([
    "и", "в", "на", "с", "для", "по", "к", "из", "о", "за", "от", "кто", "что", "это", "так", "как", "такой", "не", "да", "же", "может", "быть", "я"
])

def preprocess_text(text):
    tokenizer = WordPunctTokenizer()
    text = text.lower()  
    text = re.sub(r'[^a-zа-яё\s]', '', text)  
    tokens = tokenizer.tokenize(text)  

    tokens = [token for token in tokens if token not in stop_words]

    return tokens

data['processed_text'] = data['comment'].apply(preprocess_text)

In [5]:
model = Word2Vec(data['processed_text'], 
                 vector_size=64,       
                 window=5,             
                 min_count=5,          
                 sg=0)                 


In [6]:
def get_text_vector(tokens, model):
    word_vectors = [model.wv[token] for token in tokens if token in model.wv]
    
    if len(word_vectors) == 0:
        return None#np.zeros(model.vector_size) 
    return np.mean(word_vectors, axis=0)

data['text_vector'] = data['processed_text'].apply(lambda x: get_text_vector(x, model))
data = data.dropna(subset=['text_vector'])

In [7]:
X = np.array(data['text_vector'].tolist()) 
y = data['toxic'] 

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf1 = RandomForestClassifier(class_weight='balanced')
clf2 = LogisticRegression(solver='liblinear', class_weight='balanced')
clf3 = SVC(kernel='linear', class_weight='balanced', probability=True)

clf1.fit(X_train, y_train)
clf2.fit(X_train, y_train)
clf3.fit(X_train, y_train)

voting_clf = VotingClassifier(estimators=[('rand', clf1), ('lr', clf2), ('SVC', clf3)], voting='soft')

voting_clf.fit(X_train, y_train)

y_pred = voting_clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.73      0.92      0.82      2838
         1.0       0.69      0.35      0.46      1446

    accuracy                           0.73      4284
   macro avg       0.71      0.63      0.64      4284
weighted avg       0.72      0.73      0.70      4284



In [8]:
simple_test = 'любовь'
#'любовь это самое главное в нашей жизни'
#'хватит лежать'
#'это наш общий ребенок'

tokens = preprocess_text(simple_test)  
text_vector = get_text_vector(tokens, model) 

y_pred = clf1.predict([text_vector])
y_pred[0]

0.0

In [9]:
word_vectors = [model.wv[token] for token in data['processed_text'][0] if token in model.wv]

In [10]:
#BOW
data['processed_text_str'] = data['processed_text'].apply(lambda x: ' '.join(x))

vectorizer = CountVectorizer()
y = data['toxic'] 
X = vectorizer.fit_transform(data['processed_text_str'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

clf = SVC(kernel='linear', class_weight='balanced', probability=True)#RandomForestClassifier(class_weight='balanced', random_state=42)

clf.fit(X_train, y_train)

y_pred = clf.predict(X_test)

print(classification_report(y_test, y_pred))

              precision    recall  f1-score   support

         0.0       0.86      0.92      0.89      2838
         1.0       0.82      0.71      0.76      1446

    accuracy                           0.85      4284
   macro avg       0.84      0.82      0.83      4284
weighted avg       0.85      0.85      0.85      4284



In [11]:
simple_test = 'это наш общий ребенок'#'хватит лежать'#'любовь это самое главное в нашей жизни'
#'хватит лежать'
#'это наш общий ребенок'
processed_input = preprocess_text(simple_test)

processed_input_str = ' '.join(processed_input)

X_user = vectorizer.transform([processed_input_str])

clf.predict(X_user)[0]

1.0

In [12]:
data.head()

,comment,toxic,processed_text,text_vector,processed_text_str
0,"Верблюдов-то за что? Дебилы, бл...\n",1.0,"[верблюдовто, дебилы, бл]","[0.20458055, -0.20117, 0.09967521, 0.06201748,...",верблюдовто дебилы бл
1,"Хохлы, это отдушина затюканого россиянина, мол...",1.0,"[хохлы, отдушина, затюканого, россиянина, мол,...","[0.76204073, -0.75936, 0.5044583, 0.28179464, ...",хохлы отдушина затюканого россиянина мол вон а...
2,Собаке - собачья смерть\n,1.0,"[собаке, собачья, смерть]","[0.1356669, -0.13263024, 0.06666818, 0.0511709...",собаке собачья смерть
3,"Страницу обнови, дебил. Это тоже не оскорблени...",1.0,"[страницу, обнови, дебил, тоже, оскорбление, а...","[0.74406666, -0.76515055, 0.4315863, 0.2039924...",страницу обнови дебил тоже оскорбление а доказ...
4,"тебя не убедил 6-страничный пдф в том, что Скр...",1.0,"[тебя, убедил, страничный, пдф, том, скрипалей...","[0.61351424, -0.6146099, 0.382457, 0.12391656,...",тебя убедил страничный пдф том скрипалей отрав...


In [14]:
with open('result.txt', 'w') as f:
    f.write('test')

In [16]:
!cat result.txt

test